In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
import numpy as np
import time

# Iniciar random
np.random.seed(6)

# Variables globales con información para acelerar procesamiento
g_filas = g_cols   = 0    # Serán números (1.000.000 y 11 en este caso)
g_pesos = g_dpesos = None # Serán arrays de 11 elementos
g_sesgo = g_dsesgo = 0    # Serán números (coma flotante)

ITER = 40
APRN = 0.5
LMBD = 0.000002

# Funciones

In [2]:
# linea es un string de la forma:
# "n0, n1, n2, ..., n10, r" donde nX es una propiedad del tráfico y r es
# su clasificación (1 si es botnet, 0 si no lo es)
def formato_inicial(linea):
    aux   = [float(i) for i in linea.split(',')]
    datos = np.array(aux[:-1])
    resul = aux[-1]
    return (datos, resul)

# Devuelve un RDD de los datos del fichero fn. La lectura con Spark
# ya es mucho más rápida que leyendo con numpy.loadtxt()
def readFile(fn):
    global g_filas, g_cols
    sc      = SparkContext("local[*]", "BotnetParalelaPruebas")
    todo    = sc.textFile(fn)
    ret     = todo.map(formato_inicial)
    g_filas = todo.count()
    g_cols  = len(ret.take(1)[0][0])
    return ret

In [3]:
def norm_tupla(tuplaXy):
    media = np.mean(tuplaXy[0])
    stdev = np.std(tuplaXy[0])
    tnorm = (tuplaXy[0] - media)/stdev
    return (tnorm, tuplaXy[1])

# Toma por parámetro un RDD de 1.000.000 de registros.
# Cada registro tiene la forma (X, y)
# X es un array con 11 flotantes
# y es 1 ó 0
# Devuelve un RDD equivalente donde la X de cada tupla está
# reescalada a N(0, 1) (media = 0, desv. est. = 1)
def normalizar(rdd):
    return rdd.map(norm_tupla)

In [4]:
# fila: (X, y, ŷ)
# Devuelve un array de 11 medias
def formato_deriv(fila):
    aux = (fila[2] - fila[1])
    return fila[0] * aux

# g_pesos: Array de (1Mx1)
# g_sesgo: flotante
# datos: RDD de 1M de regs. Cada reg: tupla (X, y) donde X es un array de (11x1)
# datos x sesgo: Array de (1Mx1)
# La multiplicación se puede hacer por filas y no se altera el resultado
# El resultado es que datos ahora es el mismo RDD pero las tuplas son (X, y, ŷ),
#  donde ŷ es la predicción de resultado
def sigmoide(datos):
    global g_pesos, g_sesgo
    entrada = datos.map(lambda fila: (fila[0], fila[1], np.matmul(fila[0], g_pesos) + g_sesgo))
    salida  = entrada.map(lambda fila: (fila[0], fila[1], 1/(1 + np.exp(-fila[2]))))
    return salida

def entrenar(datos, iteraciones, aprendizaje, tasareg):
    global g_pesos, g_dpesos
    global g_sesgo, g_dsesgo
    f = g_filas # 1.000.000
    c = g_cols  # 11
    g_pesos  = np.random.random([c, ])
    g_dpesos = np.zeros([c, ])
#   for it in range(iteraciones): # (las siguientes dos líneas irían dentro del for)
    preds  = sigmoide(datos) # RDD 1M (X, y, ŷ)
    derivs = preds.map(lambda fila: (fila[2] - fila[1]) * fila[0]) # (ŷ - y) * X
    # Calcular la derivada del sesgo: 1/f*np.sum(pred_res-res)
    # Asignar pesos y sesgo:
    #    w  = w - (aprendizaje * dw)
    #    b  = b - (aprendizaje * db)
    # Calcular función de coste en cada iteración
    return derivs

# Pruebas

In [5]:
t1  = time.time()
rdd = readFile("../../../datos/botnet_tot_syn_l.csv")
t2  = time.time()
print("Tiempo transcurrido: {} segundos.".format(t2 - t1))

Tiempo transcurrido: 3.781414270401001 segundos.


In [6]:
t1 = time.time()
datos = normalizar(rdd)
t2 = time.time()
print("Tiempo transcurrido: {} segundos.".format(t2 - t1))
# Haz la media de uno de los X. Tiene que ser ~0
# print(np.mean(datos.take(1)[0][0])) # 0.000000000000000010092936587501423
# La desviación estándar tiene que ser ~1
# print(np.std(datos.take(1)[0][0])) # 1.0

# 1.2345e-05 = 0.000012345

Tiempo transcurrido: 0.0 segundos.


In [7]:
t1 = time.time()
derivadas = entrenar(datos, ITER, APRN, LMBD)
t2 = time.time()
print("Tiempo transcurrido: {} segundos.".format(t2 - t1))

Tiempo transcurrido: 9.655952453613281e-05 segundos.


In [8]:
# derivs
print(type(derivadas))
print("1000000") # aux.count())
print(derivadas.take(1))

<class 'pyspark.rdd.PipelinedRDD'>
1000000
[array([ 0.19724551,  0.19724478,  0.19724551,  0.19724552,  0.19724549,
        0.19724549,  0.19724551,  0.19724552,  0.19724543, -0.88760373,
       -0.88760504])]


In [9]:
t1 = time.time()
red = derivadas.reduce(lambda x, y: x + y)
t2 = time.time()
print("Tiempo transcurrido: {} segundos.".format(t2 - t1))

Tiempo transcurrido: 16.022749662399292 segundos.


In [10]:
t1 = time.time()
derivadas_final = red / g_filas
t2 = time.time()
print("Tiempo transcurrido: {} segundos.".format(t2 - t1))
print(derivadas_final)

Tiempo transcurrido: 5.364418029785156e-05 segundos.
[-0.02200581 -0.02043231 -0.02322289 -0.02407498  0.02178561 -0.03084179
 -0.0243429  -0.02434328 -0.02420097  0.17837562 -0.0066963 ]
